In [ ]:
# Fuzzy Annuity Valuation

def discount_value(amount, t, r):
    """
    Discounts a value (or interval) 'amount' at time 't' using rate 'r'.
    'amount' can be:
      - float (discrete value)
      - (float, float) (interval)
    """
    factor = (1 + r) ** t
    if isinstance(amount, tuple):
        return (amount[0] / factor, amount[1] / factor)
    else:
        return amount / factor

def add_values(v1, v2):
    """
    Adds two values that can be discrete or intervals.
    v1 and v2 can be:
       - float (discrete)
       - (float, float) (interval)
    Returns a discrete value or interval.
    """
    if isinstance(v1, tuple) and isinstance(v2, tuple):
        # Interval + Interval: [a,b] + [c,d] = [a+c, b+d]
        return (v1[0] + v2[0], v1[1] + v2[1])
    elif isinstance(v1, tuple) and not isinstance(v2, tuple):
        # Interval + Discrete
        return (v1[0] + v2, v1[1] + v2)
    elif not isinstance(v1, tuple) and isinstance(v2, tuple):
        # Discrete + Interval
        return (v1 + v2[0], v1 + v2[1])
    else:
        # Both discrete
        return v1 + v2

def compute_pv_annuity(D, n, r):
    """
    Computes the Present Value (PV) set of a temporal constant hesitant fuzzy annuity.
    The annuity is defined as n periods, each with the same hesitant financial capital D.
    D is a list of elements: either floats (discrete) or ('interval', a, b).
    n is the number of periods.
    r is the discount rate.

    The PV set is composed of all possible combinations of elements from D over n periods,
    discounted accordingly.
    Times are assumed to be t_i = i for i = 1 to n (1-based indexing).
    """
    from itertools import product

    # Compute discount factors
    discount_factors = [(1 / ((1 + r) ** i)) for i in range(1, n + 1)]

    # Prepare discounted values for each element in D at each time
    # We will have a list of lists, each element represents PV possibilities for a single period
    pv_each_period = []
    for i, v in enumerate(discount_factors, start=1):
        # Discount each element of D at time i
        period_values = []
        for elem in D:
            if isinstance(elem, tuple) and elem[0] == 'interval':
                discounted = discount_value((elem[1], elem[2]), i, r)
            else:
                discounted = discount_value(elem, i, r)
            period_values.append(discounted)
        pv_each_period.append(period_values)

    # Now compute all combinations (Cartesian product) to form the total PV
    total_pvs = []
    for combo in product(*pv_each_period):
        # combo is a tuple with one discounted value from each period
        current_sum = combo[0]
        for nxt in combo[1:]:
            current_sum = add_values(current_sum, nxt)
        total_pvs.append(current_sum)

    return total_pvs

def format_hesitant_set(h_set):
    """
    Formats a hesitant set that may contain discrete values and intervals.
    h_set: list of floats and/or (float, float) tuples.
    Returns a string representing the union of all elements.
    """
    def sort_key(x):
        if isinstance(x, tuple):
            return x[0]
        else:
            return x

    # Remove duplicates
    unique_values = []
    for val in h_set:
        if val not in unique_values:
            unique_values.append(val)
    unique_values.sort(key=sort_key)

    parts = []
    for val in unique_values:
        if isinstance(val, tuple):
            parts.append(f"[{val[0]:.2f}, {val[1]:.2f}]")
        else:
            parts.append(f"{{{val:.2f}}}")
    return " ∪ ".join(parts)


if __name__ == "__main__":
    # Example: Compute PV for a given annuity A from the text
    # Annuity A:
    # D = {1000} ∪ [1100,1200], n = 3, r = 0.10
    D_A = [1000, ('interval', 1100, 1200)]
    n = 3
    r = 0.10
    pv_A = compute_pv_annuity(D_A, n, r)
    print("PV(A) =", format_hesitant_set(pv_A))


PV(A) = {2486.85} ∪ [2561.98, 2637.11] ∪ [2569.50, 2652.14] ∪ [2577.76, 2668.67] ∪ [2644.63, 2802.40] ∪ [2652.89, 2818.93] ∪ [2660.41, 2833.96] ∪ [2735.54, 2984.22]


In [ ]:
# Comparing fuzzy annuities
def min_value_of_set(h_set):
    """
    Returns the minimum possible value from a hesitant set of values.
    Each element can be:
      - float (discrete)
      - ('interval', min_val, max_val)
    """
    mins = []
    for val in h_set:
        if isinstance(val, tuple) and val[0] == 'interval':
            # Use the minimum of the interval
            mins.append(val[1])
        else:
            # Discrete value
            mins.append(val)
    return min(mins) if mins else None

def max_value_of_set(h_set):
    """
    Returns the maximum possible value from a hesitant set of values.
    Each element can be:
      - float (discrete)
      - ('interval', min_val, max_val)
    """
    maxs = []
    for val in h_set:
        if isinstance(val, tuple) and val[0] == 'interval':
            # Use the maximum of the interval
            maxs.append(val[2])
        else:
            # Discrete value
            maxs.append(val)
    return max(maxs) if maxs else None

def set_difference(A, B):
    """
    Computes A \ B for two sets A and B of discrete values and/or intervals.
    Here we treat A and B as sets of distinct elements.
    """
    A_diff = []
    for a in A:
        if a not in B:
            A_diff.append(a)
    return A_diff

def all_elements_less(X, Y):
    """
    Checks if for all x in X and all y in Y, x < y.
    We use a simplifying condition: max(X) < min(Y).
    If Y is empty, return False (no y to compare).
    If X is empty and Y not empty, return True (vacuously true).
    """
    if not Y:
        return False
    if not X:
        return True
    X_max = max_value_of_set(X)
    Y_min = min_value_of_set(Y)
    if X_max is None or Y_min is None:
        return False
    return X_max < Y_min

def leq_symmetric_order(A, B):
    """
    Checks if A ≤_0 B under the symmetric order.
    A ≤_0 B if:
      (A < B\A) and (A\B < B)
    """
    B_minus_A = set_difference(B, A)
    A_minus_B = set_difference(A, B)

    cond1 = all_elements_less(A, B_minus_A)
    cond2 = all_elements_less(A_minus_B, B)
    return cond1 and cond2

def compare_fuzzy_annuities(PV_A, PV_B):
    """
    Given two PV sets of annuities, compare them using the symmetric order.
    """
    A_leq_B = leq_symmetric_order(PV_A, PV_B)
    B_leq_A = leq_symmetric_order(PV_B, PV_A)

    if A_leq_B and not B_leq_A:
        return "Annuity A is preferred over Annuity B."
    elif B_leq_A and not A_leq_B:
        return "Annuity B is preferred over Annuity A."
    elif A_leq_B and B_leq_A:
        return "A and B are equivalent under the symmetric order."
    else:
        return "Neither annuity is preferred; they cannot be directly compared."

if __name__ == "__main__":
    # Example sets:
    PV_A = [2486.85, ('interval', 2561.99, 2984.23)]
    PV_B = [2362.50, ('interval', 2512.27, 3108.56)]

    result = compare_fuzzy_annuities(PV_A, PV_B)
    print(result)


Neither annuity is preferred; they cannot be directly compared.
